In [1]:
import pandas as pd
import numpy as np

In [2]:
import osmnx as ox

In [3]:
G = ox.graph_from_place('Slovakia', network_type='drive_service',
                        simplify=True, retain_all=False)

In [7]:
'''
import pickle
with open('../data/oxmnx_graph_slovakia', 'wb') as f:
    pickle.dump(G,f)
    f.close()
'''

In [4]:
hwy_speeds = {'motorway':130,
              'trunk':130,
              'primary':90,
              'secondary':90,
              'motorway_link':90,
              'trunk_link':90,
              'primary_link':90,
              'secondary_link':90,
              'escape':90,
              'tertiary':60, 
              'residential':50,
              'tertiary_link':50,
              'living_street':60}

In [8]:
G = ox.add_edge_speeds(G, hwy_speeds,fallback=50)

In [9]:
G=ox.speed.add_edge_travel_times(G, precision=1)

In [16]:
data_munic=pd.read_excel('OBCE_2011_2GPS.xlsx').loc[:,['OBEC_LAU2','latitude_obec','longitude_obec']]

In [24]:
matrix_munic=pd.DataFrame(columns=data_munic.OBEC_LAU2.astype(str).rename(None),index=data_munic.OBEC_LAU2.astype(str).rename(None))

In [36]:
matrix_munic

,528595,529311,529320,529338,529346,529354,529362,529401,529371,529389,...,543977,543985,543993,544001,544019,544027,544035,544043,580050,599981
528595,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529311,646.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529320,458.8,298.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529338,633.5,217,229.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529346,306.8,536.5,332.9,474,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
580050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
import time

In [ ]:
for i in range(matrix_munic.shape[0]):
    print(i)
    start = time.time()
    for j in range(matrix_munic.shape[0]):
        if i>j:
            a=ox.distance.get_nearest_node(G, data_munic.loc[i,['latitude_obec','longitude_obec']].tolist(), method='haversine', return_dist=False)
            b=ox.distance.get_nearest_node(G, data_munic.loc[j,['latitude_obec','longitude_obec']].tolist(), method='haversine', return_dist=False)
            route = ox.shortest_path(G, a, b,weight='travel_time')
            matrix_munic.iloc[i,j]=sum([x['travel_time'] for x in ox.utils_graph.get_route_edge_attributes(G,route)])
    
    end = time.time()
    print(end - start)

0
0.00023102760314941406
1
0.9615271091461182
2
1.31758713722229
3
2.028771162033081
4
2.869291067123413
5
3.4276912212371826
6
3.8479816913604736
7
4.68025803565979
8
7.666805028915405
9
5.9494099617004395
10
6.523220062255859
11
7.253939867019653
12
7.864399194717407
13
8.711182117462158
14
9.14397406578064
15
11.18546199798584
16
10.452440977096558
17
11.436620950698853
18
12.607847929000854
19
14.278484344482422
20
13.919026136398315
21
14.334827899932861
22
16.918799877166748
23
15.488898992538452
24
17.068580150604248
25
19.387409925460815
26
18.39744520187378
27
18.515371799468994
28
21.567591905593872
29
22.51055073738098
30
26.22249484062195
31
21.066810846328735
32
23.84663701057434
33
25.165855169296265
34
23.487043857574463
35
23.967745065689087
36
26.716181993484497
37
25.718146085739136
38
26.993295907974243
39
26.385263919830322
40
27.972230911254883
41
29.701815128326416
42


In [ ]:
matrix_munic.to_excel('./municipality_distances.xlsx')